In [1]:
import openpyxl
import pandas as pd

xlsxFile = '19.01_20.10메뉴표.xlsx'
sheetList = []

wb = openpyxl.load_workbook(xlsxFile)
for i in wb.sheetnames:
    sheetList.append(i)

xlsx = pd.ExcelFile(xlsxFile)

for j in sheetList:
    df = pd.read_excel(xlsx, j)

In [44]:
menu2019_sheet=[]
for i in range(len(sheetList)):
    menu2019_sheet.append(pd.read_excel('19.01_20.10메뉴표.xlsx',index_col=0,sheet_name=sheetList[i]).iloc[[1,2,4,11,12,13,14,15,17,18,20,22,24],0:7])  

In [46]:
for i in range(len(sheetList)):
    menu2019_sheet[i].columns=menu2019_sheet[i].iloc[0,:]
    menu2019_sheet[i]=menu2019_sheet[i].iloc[1:,:]
menu2019_sheet     

[nan                               월(1/7)       화(1/8)     수(1/9)    목(1/10)  \
 주  간  메  뉴  표                                                                 
 조 식                                  컵라면          육개장    건새우시금치국      순두부백탕   
 NaN                            스크램블에그*케찹     손만두찜*초간장      꼬막양념찜   오징어숙회*초장   
 중 식/야 식                           생일자의 날        표고계란국      청국장찌개     돈육김치찌개   
 NaN                                 조기구이          NaN        NaN   돼지고기:국내산   
 NaN                               조기:기니산         낙지볶음     돈육깻잎볶음   닭가슴살칠리볶음   
 NaN                                 버섯잡채      낙지:베트남산   돼지고기:국내산    닭고기:국내산   
 NaN                                 봄동나물       두부양념구이   해물동그랑땡조림  꼴뚜기꽈리고추조림   
 NaN                               꽃맛살샐러드        콩나물무침     물미역*초장      시금치나물   
 NaN                               북어채미역국     글레이즈드도너츠       꿀레몬차        율무차   
 석 식                               얼갈이된장국        매생이굴국     콩가루배추국       된장찌개   
 NaN                                 안동찜

In [48]:
menu=[]  
for i in range(len(menu2019_sheet)):
    menu.append(menu2019_sheet[i].T)
menu    

[주  간  메  뉴  표                      조 식        NaN  중 식/야 식        NaN  \
 nan                                                                     
 월(1/7)                             컵라면  스크램블에그*케찹    생일자의 날      조기구이   
 화(1/8)                             육개장   손만두찜*초간장     표고계란국       NaN   
 수(1/9)                         건새우시금치국      꼬막양념찜     청국장찌개       NaN   
 목(1/10)                          순두부백탕   오징어숙회*초장    돈육김치찌개  돼지고기:국내산   
 금(1/11)                          다시마무국    돈사태단호박찜  홍합나가사키짬뽕       NaN   
 토(1/12)                           흑임자죽     잡채어묵볶음    북어콩나물국       NaN   
 일(1/13)                            NaN        NaN     두부김치국       NaN   
 
 주  간  메  뉴  표                       NaN       NaN        NaN     NaN  \
 nan                                                                    
 월(1/7)                           조기:기니산      버섯잡채       봄동나물  꽃맛살샐러드   
 화(1/8)                             낙지볶음   낙지:베트남산     두부양념구이   콩나물무침   
 수(1/9)                           돈육깻잎볶음

In [52]:
for i in range(len(menu2019_sheet)):
    menu[i].columns=[ '조식',  '조식', '중식',  '중식',  '중식',  '중식',  '중식',  '중식', '중식',  '석식', '석식', '석식']    
    menu[i]=menu[i]

In [59]:
for i in range(len(menu)):
    menu[i]=menu[i].reset_index()

In [61]:
for i in range(len(menu)):
    menu[i]=menu[i].rename(columns={'index':'식단구분'})

In [55]:
import numpy as np

def gather( df, key, value, cols ):
    id_vars = [ col for col in df.columns if col not in cols ]
    id_values = cols
    var_name = key
    value_name = value
    return pd.melt( df, id_vars, id_values, var_name, value_name )

In [65]:
for i in range(len(menu)): 
    menu[i] = gather( menu[i], 'date','menu', menu[i].columns.values[1:])

In [69]:
for i in range(1,len(menu)):
    menu[0]=pd.concat([menu[0],menu[i]])
menu[0]

,식단구분,date,menu
0,조식,월(1/7),컵라면
1,조식,월(1/7),스크램블에그*케찹
2,중식,월(1/7),생일자의 날
3,중식,월(1/7),조기구이
4,중식,월(1/7),조기:기니산
...,...,...,...
79,중식,2020-11-01 00:00:00,NaN
80,중식,2020-11-01 00:00:00,포기김치
81,석식,2020-11-01 00:00:00,된장찌개
82,석식,2020-11-01 00:00:00,오징어콩나물찜


In [75]:
import re
# menu[0].columns=['조/중/석', 'timestamp', 'menu']
menu[0]['menu']=menu[0]['menu'].apply(lambda x : str(x).rsplit('\n',1)[0])
menu[0]['국']='.'
menu[0]['국']=menu[0]['menu'].apply(lambda x : re.findall('[가-힣]+찌개$|[가-힣]+국$|[가-힣]+탕$',str(x)))
menu[0]['국']=menu[0]['국'].apply(lambda x : str(x).replace('[','').replace(']',''))

In [88]:
menu[0]=menu[0][~menu[0].menu.str.contains(':')]

In [97]:
menu[0]=menu[0][~menu[0].menu.str.contains('포기김치|열무김치|맛김치|배추김치|쌀밥|깍두기')]

In [144]:
menu[0]=menu[0][~menu[0].menu.str.contains('의 날|데 이')]

In [194]:
menu[0]=menu[0].reset_index().drop('index',axis=1)

In [128]:
menu[0]=menu[0][menu[0]['menu']!='nan']

In [208]:
menu[0]['timestamp'][:2982]=menu[0]['timestamp'][:2982].apply(lambda x : str(x).split('(',1)[1])

In [212]:
menu[0]['timestamp'][:2982]=menu[0]['timestamp'][:2982].apply(lambda x : str(x).rsplit(')',1)[0])

In [226]:
menu[0]['timestamp'][2614:2982]=menu[0]['timestamp'][2614:2982].apply(lambda x : '2020/'+str(x))

In [227]:
menu[0]['timestamp'][:2614]=menu[0]['timestamp'][:2614].apply(lambda x : '2019/'+str(x))

In [231]:
menu[0]['timestamp']=menu[0]['timestamp'].apply(lambda x : pd.to_datetime(x))

In [233]:
menu[0].to_csv('횡성메뉴.csv',index=False)

In [258]:
#!/usr/bin/env python
# coding: utf-8

# # __Same Menu 만드는 함수 만들기__
# ---

# In[ ]:


## 실행 순서
#!/usr/bin/env python
# coding: utf-8

# # __Same Menu 만드는 함수 만들기__
# ---

# In[ ]:


## 실행 순서
# MenuPreprocessing(DF, Timestamp)
# ScaledValueMaker(DF)
# SameMenuMaker(DF)


# In[1]:


import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import statistics
import re


# In[ ]:


# word1: 찾을 단어, word2: 바꿀 단어
def WordFinder(DF, find_col,change_col, row_num, word1, word2):
    if type(DF[find_col][row_num]) == str:
        if DF[find_col][row_num].find(word1) > -1:
            DF[change_col][row_num] = word2
    return 0


# In[ ]:


def MenuPreprocessing(DF, Timestamp):
    
    # Timestamp의 row_num 찾기
    row_num = list(df['timestamp']).index(Timestamp)
    
    ##################################################################    
    # ( 기준으로 데이터 쪼개기

    
    # NaN이 아니면
    if type(df['menu'][row_num]) == str :
        if df['menu'][row_num].find('(') > -1 :
            split = df['menu'][row_num].split('(')
            df['menu_nouns'][row_num] = split[0]
            df['menu_origin'][row_num] = split[1]
        else:
            df['menu_nouns'][row_num] = df['menu'][row_num]
    
    ###################################################################
    # 특수문자 지우기

    DF['menu_nouns'][row_num] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',
                                           ' ', DF['menu_nouns'][row_num])
    
    DF['menu_origin'][row_num] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',
                                       ' ', DF['menu_origin'][row_num])
    
    DF['menu_nouns'][row_num] =re.sub('양파데이|♡', '', DF['menu_nouns'][row_num])
    
    #####################################################################
    # 원산지 지우기, 재료명 통일하기
    DF['menu_origin'][row_num] = re.sub('국내산|국산|브라질산|미국산|호주산|칠레산|칠래산|독일산|외국산|뉴질랜드산|러시아산|중국산|중국|스페인|프랑스등|호주등|국내|남아공산|러시아|모로코산|벨기에산|페루산|베트남산|캐나다산|말레이시아산|사골엑기스|쇠고기엑기스|엑기스|쇠고기사골|우사골|소사골|고추분', '', DF['menu_origin'][row_num])
    DF['menu_origin'][row_num] = re.sub('우육', '쇠고기', DF['menu_origin'][row_num])
    DF['menu_origin'][row_num] = re.sub('돈육', '돼지고기', DF['menu_origin'][row_num])
    DF['menu_origin'][row_num] = re.sub('계육', '닭고기', DF['menu_origin'][row_num])
    
    #####################################################################
    # 중복된 공백, 문자열 양 옆의 공백 지우기
    
    DF['menu_noun'][row_num] = " ".join(DF['menu_noun'][row_num].split())
    DF['menu_origin'][row_num] = " ".join(DF['menu_origin'][row_num].split())
            
    #####################################################################
    # 메뉴 바꾸기
    
    
    w_list1 = ['메밀', '칼국수', '짬뽕', '우동', '냉면', '스파게티', '파스타', '짜장면', '콩나물밥', '쌈밥',  '영양밥',
           '부대찌개', '김치찌개', '짜글이', '된장찌개', '순대국', '해장국', '우거지탕','감자탕', '고추장찌개',
           '순두부', '알탕', '설렁탕', '곰탕', '닭곰탕', '육개장', '찜닭', '닭갈비', '닭볶음탕', '닭개장', '곤드레',
           '라볶이', '떡볶이', '불고기', '소불고기', '오징어불고기', '뚝배기불고기', '돈갈비', '돼지갈비', '돈육김치볶음', '돼지고기김치찜', 
           '초복', '삼계탕', '반계', '뚝배기불고기', '두루치기', '사태', '오삼불고기', '오징어볶음', '짬뽕밥',
           '카레라이스', '하이라이스', '만두국', '떡만두국', '떡국', '만두떡국', '만두전골',
           '제육볶음', '잡채밥', '볶음밥', '알밥', '모듬까스', '치킨까스', '돈까스', '돈가스', '비빔밥', '스테이크', '오므라이스', '보쌈', '돼지불백']
    
    w_list2 = ['메밀국수', '칼국수', '짬뽕', '우동', '냉면', '파스타', '파스타', '짜장면', '콩나물밥', '쌈밥',  '영양밥',
           '부대찌개', '김치찌개', '김치찌개', '된장찌개', '순대국', '해장국', '우거지탕', '감자탕', '고추장찌개', 
           '순두부찌개', '알탕', '설렁탕', '곰탕', '닭곰탕', '육개장', '찜닭', '닭갈비', '닭볶음탕', '닭개장', '곤드레밥',
           '떡볶이', '떡볶이', '불고기', '소불고기', '오징어불고기', '뚝배기불고기', '돼지갈비', '돼지갈비', '돈육김치볶음', '돼지고기김치찜',
           '삼계탕', '삼계탕', '삼계탕', '뚝배기불고기', '두루치기', '돈사태찜', '오삼불고기', '오징어볶음', '짬뽕밥',
           '카레라이스', '하이라이스', '만두국', '떡만두국', '떡국', '만두떡국', '만두국',
           '제육볶음', '잡채밥', '볶음밥', '알밥', '모듬까스', '치킨까스', '돈까스', '돈까스', '비빔밥', '스테이크', '오므라이스', '보쌈', '돼지불백']
           
    
    for num in range(len(w_list1)):
        find2change(df3, 'menu_nouns', 'menu_nouns2', row_num, w_list1[num], w_list2[num])
    
    
    #####################################################################
    # menu_nouns2가 안채워진 경우, menu_nouns 가져오기
    if type(DF['menu_nouns2'][row_num]) != str :
        DF['menu_nouns2'][row_num] = DF['menu_nouns'][row_num].split()[0]


# In[28]:


def ScaledValue(DF, start_row, length):
    DF['scaled_target'] = np.nan
    total_scaled_value = []

    #### 요일별로 scaled 값 찾기
    
    for row_num in list(range(start_row, length, 7)):
        week_list = []
        temp_scaled = []
        
        ## week_list: 한 주의 target_value를 리스트로 만들기
        ### 일반적
        if (length - start_row)%7 == 0:
            for i in range(7):
                if DF['target_value'][row_num + i] != 0 :
                    week_list.append( DF['target_value'][row_num + i] )
        ### 마지막이 7로 끊어지지 않을 때
        else:
            for i in range(len(DF)%7):
                if DF['target_value'][row_num + i] != 0 :
                    week_list.append( DF['target_value'][row_num + i] )
                

        # week_list가 빈 값이 아니면 minMaxScaler
        if week_list:
            data = np.array(week_list).reshape(-1, 1)
            minMaxScaler = MinMaxScaler()
            minMaxScaler.fit(data)
            temp_scaled = minMaxScaler.transform(data).tolist()           
        
        #### 전체 Scaled Value 리스트 만들기        
        for i in range(len(temp_scaled)):
            total_scaled_value.append(temp_scaled[i][0])
        
        #### total_scaled2 : total_scaled_value 모든 값에 + 1 하기
        # (∵ 처음부터 target_value 0인 값과 구분)
        
        total_scaled2 = []
        for i in range(len(total_scaled_value)):
            plusone = total_scaled_value[i] + 1
            total_scaled2.append(plusone)
            
    return total_scaled2      


# In[44]:


# Scaled_target 만드는 함수

def ScaledValueMaker(DF):
    length = (len(DF)//7)*7
   
    result1 = ScaledValue(DF, 0, length)
    result2 = ScaledValue(DF, length, len(DF))
    result = result1 + result2
  
    
    ### 데이터프레임 컬럼 'scaled_target'에 붙이기
    j = 0
    for row_num in range(len(DF)):
        if DF['target_value'][row_num] != 0 :
            DF['scaled_target'][row_num] = result[j]
            j = j + 1
    
    # target_value 0이어서 scaled_target NaN 나온 곳들 0으로 채우기
    DF['scaled_target'].fillna(0)


# In[45]:


def SameMenuMaker(DF):           
    ### 메뉴 & 요일이 같은 날짜 찾기
    DF['same_menu'] = np.nan
    ###### row_num1이 기준 행, row_num2이 찾는 행
    for row_num1 in range(len(DF)):
        temp_list = []
        for row_num2 in range(len(DF)):
            if ( DF['menu_nouns2'][row_num2] == DF['menu_nouns2'][row_num1] ) & ( DF['weekday'][row_num2] == DF['weekday'][row_num1] ) :
                temp_list.append(DF['scaled_target'][row_num2])
                DF['same_menu'][row_num1] = np.mean(temp_list)
                
    # 주말 (NaN) 0으로 채우기
    DF['same_menu'].fillna(0)



# In[1]:


import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import statistics
import re


# In[ ]:


# word1: 찾을 단어, word2: 바꿀 단어
def WordFinder(DF, find_col,change_col, row_num, word1, word2):
    if type(DF[find_col][row_num]) == str:
        if DF[find_col][row_num].find(word1) > -1:
            DF[change_col][row_num] = word2
    return 0


# In[ ]:


def MenuPreprocessing(DF, Timestamp):
    
    # Timestamp의 row_num 찾기
    
    row_num = list(df['timestamp']).index(Timestamp)
    
    ##################################################################    
    # ( 기준으로 데이터 쪼개기

    
    # NaN이 아니면
    if type(df['menu'][row_num]) == str :
        if df['menu'][row_num].find('(') > -1 :
            split = df['menu'][row_num].split('(')
            df['menu_nouns'][row_num] = split[0]
            df['menu_origin'][row_num] = split[1]
        else:
            df['menu_nouns'][row_num] = df['menu'][row_num]
    
    ###################################################################
    # 특수문자 지우기

    DF['menu_nouns'][row_num] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',
                                           ' ', DF['menu_nouns'][row_num])
    
    DF['menu_origin'][row_num] = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',
                                       ' ', DF['menu_origin'][row_num])
    
    DF['menu_nouns'][row_num] =re.sub('양파데이|♡', '', DF['menu_nouns'][row_num])
    
    #####################################################################
    # 원산지 지우기, 재료명 통일하기
    DF['menu_origin'][row_num] = re.sub('국내산|국산|브라질산|미국산|호주산|칠레산|칠래산|독일산|외국산|뉴질랜드산|러시아산|중국산|중국|스페인|프랑스등|호주등|국내|남아공산|러시아|모로코산|벨기에산|페루산|베트남산|캐나다산|말레이시아산|사골엑기스|쇠고기엑기스|엑기스|쇠고기사골|우사골|소사골|고추분', '', DF['menu_origin'][row_num])
    DF['menu_origin'][row_num] = re.sub('우육', '쇠고기', DF['menu_origin'][row_num])
    DF['menu_origin'][row_num] = re.sub('돈육', '돼지고기', DF['menu_origin'][row_num])
    DF['menu_origin'][row_num] = re.sub('계육', '닭고기', DF['menu_origin'][row_num])
    
    #####################################################################
    # 중복된 공백, 문자열 양 옆의 공백 지우기
    
    DF['menu_noun'][row_num] = " ".join(DF['menu_noun'][row_num].split())
    DF['menu_origin'][row_num] = " ".join(DF['menu_origin'][row_num].split())
            
    #####################################################################
    # 메뉴 바꾸기
    
    
    w_list1 = ['메밀', '칼국수', '짬뽕', '우동', '냉면', '스파게티', '파스타', '짜장면', '콩나물밥', '쌈밥',  '영양밥',
           '부대찌개', '김치찌개', '짜글이', '된장찌개', '순대국', '해장국', '우거지탕','감자탕', '고추장찌개',
           '순두부', '알탕', '설렁탕', '곰탕', '닭곰탕', '육개장', '찜닭', '닭갈비', '닭볶음탕', '닭개장', '곤드레',
           '라볶이', '떡볶이', '불고기', '소불고기', '오징어불고기', '뚝배기불고기', '돈갈비', '돼지갈비', '돈육김치볶음', '돼지고기김치찜', 
           '초복', '삼계탕', '반계', '뚝배기불고기', '두루치기', '사태', '오삼불고기', '오징어볶음', '짬뽕밥',
           '카레라이스', '하이라이스', '만두국', '떡만두국', '떡국', '만두떡국', '만두전골',
           '제육볶음', '잡채밥', '볶음밥', '알밥', '모듬까스', '치킨까스', '돈까스', '돈가스', '비빔밥', '스테이크', '오므라이스', '보쌈', '돼지불백']
    
    w_list2 = ['메밀국수', '칼국수', '짬뽕', '우동', '냉면', '파스타', '파스타', '짜장면', '콩나물밥', '쌈밥',  '영양밥',
           '부대찌개', '김치찌개', '김치찌개', '된장찌개', '순대국', '해장국', '우거지탕', '감자탕', '고추장찌개', 
           '순두부찌개', '알탕', '설렁탕', '곰탕', '닭곰탕', '육개장', '찜닭', '닭갈비', '닭볶음탕', '닭개장', '곤드레밥',
           '떡볶이', '떡볶이', '불고기', '소불고기', '오징어불고기', '뚝배기불고기', '돼지갈비', '돼지갈비', '돈육김치볶음', '돼지고기김치찜',
           '삼계탕', '삼계탕', '삼계탕', '뚝배기불고기', '두루치기', '돈사태찜', '오삼불고기', '오징어볶음', '짬뽕밥',
           '카레라이스', '하이라이스', '만두국', '떡만두국', '떡국', '만두떡국', '만두국',
           '제육볶음', '잡채밥', '볶음밥', '알밥', '모듬까스', '치킨까스', '돈까스', '돈까스', '비빔밥', '스테이크', '오므라이스', '보쌈', '돼지불백']
           
    
    for num in range(len(w_list1)):
        find2change(df3, 'menu_nouns', 'menu_nouns2', row_num, w_list1[num], w_list2[num])
    
    
    #####################################################################
    # menu_nouns2가 안채워진 경우, menu_nouns 가져오기
    if type(DF['menu_nouns2'][row_num]) != str :
        DF['menu_nouns2'][row_num] = DF['menu_nouns'][row_num].split()[0]


# In[28]:


def ScaledValue(DF, start_row, length):
    DF['scaled_target'] = np.nan
    total_scaled_value = []

    #### 요일별로 scaled 값 찾기
    
    for row_num in list(range(start_row, length, 7)):
        week_list = []
        temp_scaled = []
        
        ## week_list: 한 주의 target_value를 리스트로 만들기
        ### 일반적
        if (length - start_row)%7 == 0:
            for i in range(7):
                if DF['target_value'][row_num + i] != 0 :
                    week_list.append( DF['target_value'][row_num + i] )
        ### 마지막이 7로 끊어지지 않을 때
        else:
            for i in range(len(DF)%7):
                if DF['target_value'][row_num + i] != 0 :
                    week_list.append( DF['target_value'][row_num + i] )
                

        # week_list가 빈 값이 아니면 minMaxScaler
        if week_list:
            data = np.array(week_list).reshape(-1, 1)
            minMaxScaler = MinMaxScaler()
            minMaxScaler.fit(data)
            temp_scaled = minMaxScaler.transform(data).tolist()           
        
        #### 전체 Scaled Value 리스트 만들기        
        for i in range(len(temp_scaled)):
            total_scaled_value.append(temp_scaled[i][0])
        
        #### total_scaled2 : total_scaled_value 모든 값에 + 1 하기
        # (∵ 처음부터 target_value 0인 값과 구분)
        
        total_scaled2 = []
        for i in range(len(total_scaled_value)):
            plusone = total_scaled_value[i] + 1
            total_scaled2.append(plusone)
            
    return total_scaled2      


# In[44]:


# Scaled_target 만드는 함수

def ScaledValueMaker(DF):
    length = (len(DF)//7)*7
   
    result1 = ScaledValue(DF, 0, length)
    result2 = ScaledValue(DF, length, len(DF))
    result = result1 + result2
  
    
    ### 데이터프레임 컬럼 'scaled_target'에 붙이기
    j = 0
    for row_num in range(len(DF)):
        if DF['target_value'][row_num] != 0 :
            DF['scaled_target'][row_num] = result[j]
            j = j + 1
    
    # target_value 0이어서 scaled_target NaN 나온 곳들 0으로 채우기
    DF['scaled_target'].fillna(0)


# In[45]:


def SameMenuMaker(DF):           
    ### 메뉴 & 요일이 같은 날짜 찾기
    DF['same_menu'] = np.nan
    ###### row_num1이 기준 행, row_num2이 찾는 행
    for row_num1 in range(len(DF)):
        temp_list = []
        for row_num2 in range(len(DF)):
            if ( DF['menu_nouns2'][row_num2] == DF['menu_nouns2'][row_num1] ) & ( DF['weekday'][row_num2] == DF['weekday'][row_num1] ) :
                temp_list.append(DF['scaled_target'][row_num2])
                DF['same_menu'][row_num1] = np.mean(temp_list)
                
    # 주말 (NaN) 0으로 채우기
    DF['same_menu'].fillna(0)



In [241]:
a=pd.read_csv('횡성메뉴.csv')
b=pd.read_csv('횡성 input1.csv')

In [242]:
a

,조/중/석,timestamp,menu,국
0,조식,2019-01-07,컵라면,NaN
1,조식,2019-01-07,스크램블에그*케찹,NaN
2,중식,2019-01-07,조기구이,NaN
3,중식,2019-01-07,버섯잡채,NaN
4,중식,2019-01-07,봄동나물,NaN
...,...,...,...,...
4423,중식,2020-11-01,스모크햄전*케찹,NaN
4424,중식,2020-11-01,부들어묵볶음,NaN
4425,석식,2020-11-01,된장찌개,'된장찌개'
4426,석식,2020-11-01,오징어콩나물찜,NaN


In [245]:
data=pd.merge(a,b,on='timestamp',how='inner')

In [261]:
for num in range(len(data)):
    timestamp = data.iloc[num, 1]
    MenuPreprocessing(data, timestamp)
ScaledValueMaker(data)
# SameMenuMaker(data)

KeyError: 'timestamp'

In [254]:
data.iloc[10, 1]

'2019-01-08'

In [253]:
data

,조/중/석,timestamp,menu,국,target_value,dayofweek_mean,dayofweek_median,weekday,temperature,abnormal,rainfall,holiday,before_holiday,after_holiday,출근인원,scaled_target,same_menu
0,조식,2019-01-07,컵라면,NaN,156,153,158,0,3.1,0,0.0,0,0,0,139,1.0,NaN
1,조식,2019-01-07,스크램블에그*케찹,NaN,156,153,158,0,3.1,0,0.0,0,0,0,139,1.0,NaN
2,중식,2019-01-07,조기구이,NaN,156,153,158,0,3.1,0,0.0,0,0,0,139,1.0,NaN
3,중식,2019-01-07,버섯잡채,NaN,156,153,158,0,3.1,0,0.0,0,0,0,139,1.0,NaN
4,중식,2019-01-07,봄동나물,NaN,156,153,158,0,3.1,0,0.0,0,0,0,139,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4418,중식,2020-10-31,삼치카레구이,NaN,45,42,41,5,18.2,1,0.0,0,0,0,35,1.0,NaN
4419,중식,2020-10-31,메추리알장조림,NaN,45,42,41,5,18.2,1,0.0,0,0,0,35,1.0,NaN
4420,석식,2020-10-31,얼갈이된장국,'얼갈이된장국',45,42,41,5,18.2,1,0.0,0,0,0,35,1.0,NaN
4421,석식,2020-10-31,두부스크램블*케찹,NaN,45,42,41,5,18.2,1,0.0,0,0,0,35,1.0,NaN
